# NBA Game Log Data Analysis

Analysis of 523,825 NBA game logs from 2003-2024, focusing on data quality and statistical validation. Key areas examined: team names standardization, location/outcome encoding, and points calculation verification.

## Data Loading
Loading 20 seasons of game logs from CSV files, with date parsing and sorting.

In [16]:
import pandas as pd
from pathlib import Path

# Combine all season files
seasons_data = []
raw_dir = Path("../data/archive/raw")

for file in raw_dir.glob("game_logs_*.csv"):
    df = pd.read_csv(file)
    df['season'] = int(file.stem.split('_')[2])  # Extract year from filename
    seasons_data.append(df)

df = pd.concat(seasons_data, ignore_index=True)

# type conversion
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

In [17]:
# set options to see all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Display basic dataset info

In [18]:
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
print(df.dtypes)

Dataset Shape: (539777, 26)

Column Names:
['date', 'team', 'location', 'opponent', 'outcome', 'active', 'seconds_played', 'made_field_goals', 'attempted_field_goals', 'made_three_point_field_goals', 'attempted_three_point_field_goals', 'made_free_throws', 'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points_scored', 'game_score', 'plus_minus', 'player_id', 'name', 'season']

Data Types:
date                                 datetime64[ns]
team                                         object
location                                     object
opponent                                     object
outcome                                      object
active                                         bool
seconds_played                                int64
made_field_goals                              int64
attempted_field_goals                         int64
made_three_point_field_goals                  int64
att

### Observation:
team, location, opponent, outcome have object type

## Null and duplicate check

In [19]:
# Check null values
null_counts = df.isnull().sum()
print("Null Values:\n", null_counts[null_counts > 0])

# Check duplicates
duplicate_count = df.duplicated().sum()
print("\nDuplicate Rows:", duplicate_count)

Null Values:
 Series([], dtype: int64)

Duplicate Rows: 0


## Statistical Analysis
Examining distribution of game statistics including minutes played, scoring, rebounds, and efficiency metrics. Key findings:
- Points Per Game: Average 9.94 (Range: 0-81)
- Playing Time: Average 23.2 minutes
- Field Goal Stats: 3.69 made / 8.07 attempted

In [20]:
# Get summary statistics for numeric columns
print(df.describe())

                                date  seconds_played  made_field_goals  \
count                         539777   539777.000000     539777.000000   
mean   2014-06-19 00:07:49.633941504     1393.311516          3.696691   
min              2003-10-28 00:00:00        0.000000          0.000000   
25%              2009-01-16 00:00:00      899.000000          1.000000   
50%              2014-10-29 00:00:00     1434.000000          3.000000   
75%              2019-11-23 00:00:00     1929.000000          6.000000   
max              2025-02-04 00:00:00     3620.000000         28.000000   
std                              NaN      670.528379          3.091338   

       attempted_field_goals  made_three_point_field_goals  \
count          539777.000000                 539777.000000   
mean                8.072536                      0.836407   
min                 0.000000                      0.000000   
25%                 4.000000                      0.000000   
50%                 7.0

In [21]:
# Check unique values in categorical columns
categorical_cols = ['team', 'location', 'opponent', 'outcome']
for col in categorical_cols:
    print(f"\nUnique values in {col}:")
    print(f"Total unique values: {df[col].nunique()}")
    print(df[col].value_counts())
    print("-"*50)
    


Unique values in team:
Total unique values: 37
team
Team.SAN_ANTONIO_SPURS                    19356
Team.DALLAS_MAVERICKS                     18698
Team.UTAH_JAZZ                            18486
Team.BOSTON_CELTICS                       18353
Team.WASHINGTON_WIZARDS                   18179
Team.INDIANA_PACERS                       18169
Team.LOS_ANGELES_CLIPPERS                 18164
Team.MILWAUKEE_BUCKS                      18118
Team.MEMPHIS_GRIZZLIES                    18094
Team.GOLDEN_STATE_WARRIORS                18026
Team.ORLANDO_MAGIC                        18014
Team.ATLANTA_HAWKS                        18012
Team.DENVER_NUGGETS                       17974
Team.MINNESOTA_TIMBERWOLVES               17961
Team.TORONTO_RAPTORS                      17953
Team.SACRAMENTO_KINGS                     17912
Team.PHILADELPHIA_76ERS                   17887
Team.DETROIT_PISTONS                      17884
Team.LOS_ANGELES_LAKERS                   17879
Team.CLEVELAND_CAVALIERS           

In [22]:
# Check if points match field goals and free throws
df['calculated_points'] = (df['made_field_goals'] - df['made_three_point_field_goals']) * 2 + \
                         df['made_three_point_field_goals'] * 3 + \
                         df['made_free_throws']
points_mismatch = (df['calculated_points'] != df['points_scored']).sum()
print("\nPoints calculation mismatches:", points_mismatch)


Points calculation mismatches: 2


In [23]:
points_mismatch_rows = (df['calculated_points'] != df['points_scored'])
mismatch_rows = df[points_mismatch_rows].copy()
scoring_cols = [
   'date', 'name', 'team', 'opponent',
   'made_field_goals', 'made_three_point_field_goals', 
   'made_free_throws', 'calculated_points', 'points_scored'
]
print(mismatch_rows[scoring_cols])

            date             name                     team  \
43715 2021-01-05  Dennis Schröder  Team.LOS_ANGELES_LAKERS   
29866 2021-01-05    Anthony Davis  Team.LOS_ANGELES_LAKERS   

                     opponent  made_field_goals  made_three_point_field_goals  \
43715  Team.MEMPHIS_GRIZZLIES                 5                             1   
29866  Team.MEMPHIS_GRIZZLIES                10                             4   

       made_free_throws  calculated_points  points_scored  
43715                 0                 11             12  
29866                 3                 27             26  


## Data Quality Issues

### Team and Opponent Names
- 37 unique team values (expected: 30)
- 63 unique opponent values (expected: 30)
- Historical transitions (e.g., SuperSonics → Thunder)
- Format inconsistencies in team and opponent data

### Locations and Game Outcomes
Standardization needed for:
- Location values (HOME/AWAY formats)
- Outcome values (WIN/LOSS formats)

### Points Validation
Found 2 scoring discrepancies:
- Identified in Lakers vs Grizzlies game (2021-01-05)
- Point attribution error between Davis and Schröder
- I will omit this record due to it does not have major effect

#### Get all unique values

In [24]:
# get the list 
categorical_cols = ['location', 'outcome']
unique_values = {col: df[col].unique().tolist() for col in categorical_cols}
print("\nUnique values as lists:")
for col, values in unique_values.items():
    print(f"{col}: {values}")
    print("-"*100)


Unique values as lists:
location: ['Location.HOME', 'Location.AWAY', 'AWAY', 'HOME']
----------------------------------------------------------------------------------------------------
outcome: ['Outcome.WIN', 'Outcome.LOSS', 'LOSS', 'WIN']
----------------------------------------------------------------------------------------------------


In [25]:
categorical_cols = ['team', 'opponent']
unique_values = {col: df[col].unique().tolist() for col in categorical_cols}
all_team_names = unique_values["team"] + unique_values["opponent"]
all_team_names = sorted(set(all_team_names))
print("Unique team names occur in the data:")
print(all_team_names)

Unique team names occur in the data:
['ATLANTA HAWKS', 'BOSTON CELTICS', 'CHICAGO BULLS', 'CLEVELAND CAVALIERS', 'DALLAS MAVERICKS', 'DENVER NUGGETS', 'DETROIT PISTONS', 'GOLDEN STATE WARRIORS', 'HOUSTON ROCKETS', 'INDIANA PACERS', 'LOS ANGELES CLIPPERS', 'LOS ANGELES LAKERS', 'MEMPHIS GRIZZLIES', 'MIAMI HEAT', 'MILWAUKEE BUCKS', 'MINNESOTA TIMBERWOLVES', 'NEW JERSEY NETS', 'NEW ORLEANS HORNETS', 'NEW YORK KNICKS', 'ORLANDO MAGIC', 'PHILADELPHIA 76ERS', 'PHOENIX SUNS', 'PORTLAND TRAIL BLAZERS', 'SACRAMENTO KINGS', 'SAN ANTONIO SPURS', 'SEATTLE SUPERSONICS', 'TORONTO RAPTORS', 'Team.ATLANTA_HAWKS', 'Team.BOSTON_CELTICS', 'Team.BROOKLYN_NETS', 'Team.CHARLOTTE_BOBCATS', 'Team.CHARLOTTE_HORNETS', 'Team.CHICAGO_BULLS', 'Team.CLEVELAND_CAVALIERS', 'Team.DALLAS_MAVERICKS', 'Team.DENVER_NUGGETS', 'Team.DETROIT_PISTONS', 'Team.GOLDEN_STATE_WARRIORS', 'Team.HOUSTON_ROCKETS', 'Team.INDIANA_PACERS', 'Team.LOS_ANGELES_CLIPPERS', 'Team.LOS_ANGELES_LAKERS', 'Team.MEMPHIS_GRIZZLIES', 'Team.MIAMI_HEAT'

In [26]:
categorical_cols = ['team', 'opponent']
unique_values = {col: df[col].unique().tolist() for col in categorical_cols}
print(unique_values)

{'team': ['Team.SAN_ANTONIO_SPURS', 'Team.MIAMI_HEAT', 'Team.PHOENIX_SUNS', 'Team.DALLAS_MAVERICKS', 'Team.PHILADELPHIA_76ERS', 'Team.LOS_ANGELES_LAKERS', 'Team.MILWAUKEE_BUCKS', 'Team.WASHINGTON_WIZARDS', 'Team.ATLANTA_HAWKS', 'Team.UTAH_JAZZ', 'Team.MINNESOTA_TIMBERWOLVES', 'Team.DETROIT_PISTONS', 'Team.DENVER_NUGGETS', 'Team.CLEVELAND_CAVALIERS', 'Team.GOLDEN_STATE_WARRIORS', 'Team.PORTLAND_TRAIL_BLAZERS', 'Team.SACRAMENTO_KINGS', 'Team.TORONTO_RAPTORS', 'Team.NEW_ORLEANS_HORNETS', 'Team.NEW_YORK_KNICKS', 'Team.BOSTON_CELTICS', 'Team.NEW_JERSEY_NETS', 'Team.INDIANA_PACERS', 'Team.CHICAGO_BULLS', 'Team.ORLANDO_MAGIC', 'ATLANTA HAWKS', 'Team.HOUSTON_ROCKETS', 'Team.LOS_ANGELES_CLIPPERS', 'Team.SEATTLE_SUPERSONICS', 'Team.MEMPHIS_GRIZZLIES', 'PORTLAND TRAIL BLAZERS', 'Team.CHARLOTTE_BOBCATS', 'Team.NEW_ORLEANS_OKLAHOMA_CITY_HORNETS', 'Team.OKLAHOMA_CITY_THUNDER', 'Team.BROOKLYN_NETS', 'Team.NEW_ORLEANS_PELICANS', 'Team.CHARLOTTE_HORNETS'], 'opponent': ['Team.PHOENIX_SUNS', 'Team.PHILAD

## Check data obtained by daily API

In [27]:
from datetime import date, datetime

import pandas as pd
from nba_data_forge.etl.extractors.daily_game_log_extractor import DailyGameLogExtractor

extractor = DailyGameLogExtractor()
start_date = datetime.strptime("2024-02-01", "%Y-%m-%d")
end_date = datetime.strptime("2024-02-02", "%Y-%m-%d")
daily = extractor.extract(start_date, end_date)

2025-02-05 16:06:01 | DailyGameLogExtractor | INFO     | Extracting game logs from 2024-02-01 00:00:00 to 2024-02-02 00:00:00
2025-02-05 16:06:01 | DailyGameLogExtractor | INFO     | Skipping 2024-02-01 00:00:00 - already processed
2025-02-05 16:06:01 | DailyGameLogExtractor | INFO     | Skipping 2024-02-02 00:00:00 - already processed
2025-02-05 16:06:01 | DailyGameLogExtractor | INFO     | No new games found in date range


In [28]:
print("Dataset Shape:", daily.shape)
print("\nColumn Names:")
print(daily.columns.tolist())
print("\nData Types:")
print(daily.dtypes)
print(daily.head())

Dataset Shape: (0, 0)

Column Names:
[]

Data Types:
Series([], dtype: object)
Empty DataFrame
Columns: []
Index: []


In [29]:
categorical_cols = ['team', 'opponent', 'location', 'outcome']
unique_values = {col: daily[col].unique().tolist() for col in categorical_cols}
print(unique_values)

KeyError: 'team'

## Data Analysis Observations
### Missing Columns and Transformations Needed

1. Compared to the seasonal game logs API, the current dataset is missing:
- `Active` (boolean): Player's active status
- `points_scored` (integer): Can be derived from existing points column

### Column Mappings and Data Types

1. Player Identification:

- `slug` column corresponds to `player_id` in our schema
- Should maintain consistent `player_id` references across tables

2. Numeric Data Types:

- `plus_minus` is stored as float, which has changed from our previous obtained data

### Required Transformations

1. Team Name Standardization:

- Apply consistent team name standardization
- Create new columns for abbreviation

2. Game Information:

- Flag location to is_home (boolean)
- Flag outcome to is_win (boolean)
- Apply consistent standardization

In [15]:
from nba_data_forge.etl.transformers.game_log_transformer import GameLogTransformer

transformer = GameLogTransformer()
dailt_transformed = transformer.transform(daily)

TypeError: GameLogTransformer.transform() missing 1 required positional argument: 'filename'

In [19]:
from datetime import date, datetime

import pandas as pd
from nba_data_forge.etl.extractors.game_log_extractor import GameLogExtractor
from nba_data_forge.common.utils.paths import paths

raw_dir = paths.get_path("raw")

df_2025 = pd.read_csv(raw_dir / "game_logs_2025.csv")
print(df_2025.head())

         date                  team       location                opponent  \
0  2024-12-05  Team.NEW_YORK_KNICKS  Location.HOME  Team.CHARLOTTE_HORNETS   
1  2024-12-07  Team.NEW_YORK_KNICKS  Location.HOME    Team.DETROIT_PISTONS   
2  2024-12-09  Team.NEW_YORK_KNICKS  Location.AWAY    Team.TORONTO_RAPTORS   
3  2024-12-11  Team.NEW_YORK_KNICKS  Location.HOME      Team.ATLANTA_HAWKS   
4  2024-12-15  Team.NEW_YORK_KNICKS  Location.AWAY      Team.ORLANDO_MAGIC   

        outcome  active  seconds_played  made_field_goals  \
0   Outcome.WIN    True             735                 1   
1  Outcome.LOSS    True            1664                 2   
2   Outcome.WIN    True             974                 3   
3  Outcome.LOSS    True            1115                 1   
4   Outcome.WIN    True            1051                 1   

   attempted_field_goals  made_three_point_field_goals  \
0                      4                             0   
1                      3                        

In [20]:
print("Dataset Shape:", df_2025.shape)
print("\nColumn Names:")
print(df_2025.columns.tolist())
print("\nData Types:")
print(df_2025.dtypes)
print(df_2025.head())

Dataset Shape: (15952, 25)

Column Names:
['date', 'team', 'location', 'opponent', 'outcome', 'active', 'seconds_played', 'made_field_goals', 'attempted_field_goals', 'made_three_point_field_goals', 'attempted_three_point_field_goals', 'made_free_throws', 'attempted_free_throws', 'offensive_rebounds', 'defensive_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points_scored', 'game_score', 'plus_minus', 'player_id', 'name']

Data Types:
date                                  object
team                                  object
location                              object
opponent                              object
outcome                               object
active                                  bool
seconds_played                         int64
made_field_goals                       int64
attempted_field_goals                  int64
made_three_point_field_goals           int64
attempted_three_point_field_goals      int64
made_free_throws                       

In [21]:
categorical_cols = ['team', 'opponent', 'location', 'outcome']
unique_values = {col: df_2025[col].unique().tolist() for col in categorical_cols}
print(unique_values)

{'team': ['Team.NEW_YORK_KNICKS', 'Team.HOUSTON_ROCKETS', 'Team.MIAMI_HEAT', 'Team.TORONTO_RAPTORS', 'Team.MEMPHIS_GRIZZLIES', 'Team.DENVER_NUGGETS', 'Team.MINNESOTA_TIMBERWOLVES', 'Team.PHOENIX_SUNS', 'Team.CLEVELAND_CAVALIERS', 'Team.NEW_ORLEANS_PELICANS', 'Team.GOLDEN_STATE_WARRIORS', 'Team.MILWAUKEE_BUCKS', 'Team.ORLANDO_MAGIC', 'Team.PORTLAND_TRAIL_BLAZERS', 'Team.WASHINGTON_WIZARDS', 'Team.CHARLOTTE_HORNETS', 'Team.CHICAGO_BULLS', 'Team.LOS_ANGELES_CLIPPERS', 'Team.ATLANTA_HAWKS', 'Team.SAN_ANTONIO_SPURS', 'Team.DETROIT_PISTONS', 'Team.BROOKLYN_NETS', 'Team.PHILADELPHIA_76ERS', 'Team.BOSTON_CELTICS', 'Team.INDIANA_PACERS', 'Team.OKLAHOMA_CITY_THUNDER', 'Team.SACRAMENTO_KINGS', 'Team.LOS_ANGELES_LAKERS', 'Team.DALLAS_MAVERICKS', 'Team.UTAH_JAZZ'], 'opponent': ['Team.CHARLOTTE_HORNETS', 'Team.DETROIT_PISTONS', 'Team.TORONTO_RAPTORS', 'Team.ATLANTA_HAWKS', 'Team.ORLANDO_MAGIC', 'Team.MINNESOTA_TIMBERWOLVES', 'Team.NEW_ORLEANS_PELICANS', 'Team.SAN_ANTONIO_SPURS', 'Team.WASHINGTON_WIZ